In [ ]:
from matplotlib import pyplot 
from matplotlib.patches import *
from mtcnn.mtcnn import MTCNN
import os 
import glob 
import cv2
import csv
import pandas as pd
import re
from PIL import Image
import tensorflow as tf
import numpy as np
import re

img_dir = r"" #Image path for resized images
raw_img = r"" #Image path for raw images
crop_dir = r"" #Image path for cropped images
recrop_img = r"" #Image path for cropped and resized images

data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
raw_path = os.path.join(raw_img,'*g') 
raw = glob.glob(raw_path) 

data_path1 = os.path.join(recrop_img,'*g') 
files1 = glob.glob(data_path1) 
raw_path1 = os.path.join(crop_dir,'*g') 
raw1 = glob.glob(raw_path1) 

# resize images and store at a specific path    
def resize_images(rr,var,a,b):
	i = 0
	data = [] 
	for f1 in rr: 
		img = cv2.imread(f1) 
		res = cv2.resi=ze(img, dsize=(a, b), interpolation=cv2.INTER_CUBIC)
		path = img_dir+'\resized'+str(i)+'.jpg'
		cv2.imwrite(path,res)
		data.append(img)
		i = int(i)
		i += 1
		#print(f1)


hs = []
ws = []
xs = []
ys = []
images = []
label = []
lval = []

# draw an image with detected objects
def draw_image_with_boxes(filename, result_list):
	# load the image
	data = pyplot.imread(filename)
	# plot the image
	pyplot.imshow(data)
	# get the context for drawing boxes
	ax = pyplot.gca()
	# plot each box
	for result in result_list:
		# get coordinates
		x, y, width, height = result['box']
		# create the shape
		rect = Rectangle((x, y), width, height, fill=False, color='red')
		# draw the box
		ax.add_patch(rect)
		# show the plot
		#pyplot.show()
		d = re.findall('resized[0-9]+',filename)
		for j in d:
			label.append(j)
		images.append(filename)
		ws.append(width)
		hs.append(height)
		xs.append(x)
		ys.append(y)


resize_images(raw,img_dir,300,400)

for f1 in files: 
	# load images from filepath
	img = cv2.imread(f1)
	#print(f1)
	# create the detector, using default weights
	detector = MTCNN()
	# face detection with mtcnn on a photograph
	faces = detector.detect_faces(img)
	draw_image_with_boxes(f1, faces)
        
        
	# display faces on the original image
	#draw_image_with_boxes(f1, faces)
#print(len(xs))    
for u in range(len(xs)):
	im = Image.open(images[u])
	im1 = im.crop((xs[u],ys[u],xs[u]+ws[u],ys[u]+ws[u]))
	im1 = np.array(im1)  
	cr_path = crop_dir+'\crop'+str(u)+'.jpg'
	cv2.imwrite(cr_path,im1)

def final_resize_images():
	i = 0
	for f1 in raw1: 
		filename = str(f1).split('.jpg')[0].split('\\')[-1:][0]
		print(filename)
		if filename.find('crop')==-1:
			lval.append('0')
		else:
			lval.append('1')
		#print(f1)
		img = cv2.imread(f1) 
		res = cv2.resize(img, dsize=(10, 10), interpolation=cv2.INTER_CUBIC)
		path = crop_dir+'\resized'+str(i)+'.jpg'
		cv2.imwrite(path,res)
		i = int(i)
		i += 1

final_resize_images()
lbl = pd.DataFrame(lval)
lbl.to_csv('label.csv')
df = pd.DataFrame({'Image':images, 'X':xs,'Y':ys, 'Width':ws, 'Height':hs, 'Label':label})
df.to_csv('bound.csv')